In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [15]:
def S7_test():
    print("Test-S7: Stake is equally distributed across slots")
    num = int(input("enter the number of blocks you want to test: "))
            
    current_block = getBlockNumber()
    
    iterations = 0
    while iterations < num:
        counters = [0, 0, 0, 0]
        effect_stakes = [0.0, 0.0, 0.0, 0.0]
        print("current block", current_block)
        
        validator_infos = getAllValidatorInformation()
        total_reward = 0
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                by_shard_metrics = i['metrics']['by-shard']
                for by_shard_metric in by_shard_metrics:
                    shard_id = by_shard_metric['shard-id']
                    stake = float(by_shard_metric['effective-stake'])
                    counters[shard_id] = counters[shard_id] + 1
                    effect_stakes[shard_id] = effect_stakes[shard_id] + stake
                    
        print("counter", counters)
        print("stakes", effect_stakes)
        
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        iterations = iterations + 1
        
    print("done!")

In [16]:
S7_test()

Test-S7: Stake is equally distributed across slots
enter the number of blocks you want to test: 1
current block 10525
counter [0, 0, 0, 0]
stakes [0.0, 0.0, 0.0, 0.0]
done!
